In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Clone the repository
!git clone https://github.com/alessioborgi/StyleAlignedDiffModels.git

# Change directory to the cloned repository
%cd StyleAlignedDiffModels

# Install the required packages
!pip install -r requirements.txt > /dev/null


Cloning into 'StyleAlignedDiffModels'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 26 (delta 7), reused 11 (delta 2), pack-reused 0
Unpacking objects: 100% (26/26), 13.69 KiB | 1.14 MiB/s, done.
/kaggle/working/StyleAlignedDiffModels


In [8]:
from diffusers import StableDiffusionXLPipeline, DDIMScheduler
import torch
torch.cuda.empty_cache()
import mediapy
import sa_handler

In [ ]:
import gc
del variables
gc.collect()

In [4]:
# init models

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False,
                              set_alpha_to_one=False)
pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True,
    scheduler=scheduler
).to("cuda")

handler = sa_handler.Handler(pipeline)
sa_args = sa_handler.StyleAlignedArgs(share_group_norm=False,
                                      share_layer_norm=False,
                                      share_attention=True,
                                      adain_queries=True,
                                      adain_keys=True,
                                      adain_values=False,
                                     )

handler.register(sa_args, )

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

text_encoder/config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

text_encoder_2/config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

(…)diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

(…)diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

(…)diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# run StyleAligned
torch.cuda.memory_summary(device=None, abbreviated=False)
sets_of_prompts = [
  "a toy train. macro photo. 3d game asset",
  "a toy airplane. macro photo. 3d game asset",
  "a toy bicycle. macro photo. 3d game asset",
  "a toy car. macro photo. 3d game asset",
  "a toy boat. macro photo. 3d game asset",
]
images = pipeline(sets_of_prompts,).images
mediapy.show_images(images)

  0%|          | 0/50 [00:00<?, ?it/s]

Suggestions for Improving Results
Enhanced Attention Mechanisms:

Multi-Head Attention Optimization: Fine-tuning the number of attention heads and the way they share information could improve style consistency and reduce content leakage.
Dynamic Attention Sharing: Implementing a dynamic mechanism that adjusts the level of attention sharing based on the complexity of the input text and style might lead to better results.
Advanced Normalization Techniques:

Layer-wise AdaIN: Instead of a single AdaIN operation, applying layer-wise adaptive normalization could better capture and transfer intricate style details.
Combining Normalization Methods: Experiment with combining AdaIN with other normalization techniques like batch normalization or instance normalization to see if it enhances style transfer.
Incorporating Contextual Information:

Contextual Embeddings: Use contextual embeddings from large language models to provide additional information during the attention process, which might improve the alignment of complex styles.
Cross-Modal Attention: Integrate cross-modal attention mechanisms that consider both textual and visual contexts simultaneously, enhancing the model's ability to capture and maintain style consistency.
Architectural Modifications
Modular Attention Layers:

Plug-and-Play Attention Modules: Design modular attention layers that can be easily plugged into different parts of the network, allowing for flexible experimentation with where and how attention is shared.
Hierarchical Attention: Implement a hierarchical attention mechanism where different levels of the network focus on different aspects of style and content, ensuring a more granular alignment.
Enhanced U-Net Architecture:

Deeper U-Net: Extend the depth of the U-Net architecture used in the diffusion process to capture more detailed features at multiple scales, which could improve the richness of the generated styles.
Skip Connections with Style Information: Incorporate skip connections that carry style information directly from the input to deeper layers of the network, preserving style details more effectively.
Integration with Generative Adversarial Networks (GANs):

Hybrid Diffusion-GAN Model: Combine the diffusion model with a GAN to leverage the strengths of both architectures. The GAN could help refine the generated images and further enforce style consistency.
Discriminator Focused on Style: Train a style-focused discriminator in the GAN setup that specifically evaluates the style consistency of generated images, providing more direct feedback during training.